# Softmax regression
- a.k.a. Multinomial logistic regression

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 0. Load MNIST data

In [ ]:
mnist = input_data.read_data_sets("MNIST_idx3/", one_hot=True)

In [ ]:
X_trn, Y_trn = mnist.train.images, mnist.train.labels
X_val, Y_val = mnist.validation.images, mnist.validation.labels
X_test, Y_test = mnist.test.images, mnist.test.labels

In [ ]:
num_trn = Y_trn.shape[0]
num_val = Y_val.shape[0]
num_test = Y_test.shape[0]

print("Number of training points: ", num_trn)
print("Number of validation points: ", num_val)
print("Number of test points: ", num_test)

In [ ]:
dim_X = X_trn.shape[1]
pixel_X = int(np.sqrt(dim_X)) # np.sqrt의 출력이 float32이므로, 이를 int 자료형으로 변경
dim_Y = Y_trn.shape[1]

print("Dimension of X: %d (%d x %d)" % (dim_X, pixel_X, pixel_X))
print("Dimension of Y: ", dim_Y)

## 1. Build the graph
Tensorflow에서는 모델을 'graph'로 구현한다.

### 1.1. Calculate logits
- Placeholder for images
- Weights and biases
- Logits

$logit = b + w_1 \cdot x_1 + w_2 \cdot x_2 + ... + w_p \cdot x_p $

#### Placeholder for inputs
- Shape of the placeholder for inputs: [batch_size, input_dimension]
- Placeholder의 batch_size를 None으로 하면, placeholder에 들어가기 전에 batch size를 조절해야 함

In [ ]:
X = tf.placeholder(tf.float32, [None, dim_X])

In [ ]:
W = tf.Variable(initial_value=tf.zeros([dim_X, dim_Y]))
b = tf.Variable(initial_value=tf.zeros([dim_Y]))

In [ ]:
logits = tf.matmul(X, W) + b

In [ ]:
print(X)
print(W)
print(b)
print(logits)

#### Placeholder for inputs
- Shape of the placeholder for Y: [batch_size, output_dimension]

In [ ]:
Y = tf.placeholder(tf.float32, [None, dim_Y])

In [ ]:
print(Y)

## 2. Training

### 2.1. Loss function
- Classification 문제에서 제일 많이 사용하는 loss function은 **cross-entropy**

Cross-entropy $ = - \sum_{\substack{j}}y_j \cdot \log(y'_j)$

In [ ]:
Y_hat = tf.nn.softmax(logits)
cross_entropy_equation = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(Y_hat), reduction_indices=[1]))

In [ ]:
print(Y_hat)
print(cross_entropy_equation)

- 위처럼 수식을 입력하지 않고, **tf.nn.softmax_cross_entropy_with_logits**을 사용하는 것이 편리함

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)

In [ ]:
cost = tf.reduce_mean(cross_entropy)

In [ ]:
print(cross_entropy)

### 2.2. Training operator
- First, define the oprimizer. (**optimizer**)
- And then, define training operator. (**train_op**)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

In [ ]:
train_op = optimizer.minimize(cost)

In [ ]:
# # 다음과 같은 한 번에 많이 사용하기도 함
# train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

In [ ]:
print(optimizer)
print("==================")
print(train_op)

### 2.3. Predicting operator
- **correct_prediction**: boolean (True or False)
- **accuracy**: 먼저 correct_prediction을 float32로 변환 후에 배치 내 평균을 계산

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
# As you know, tf.argmax(logits, 1) == tf.argmax(Y_hat, 1)

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
print(correct_prediction)
print(accuracy)

### 2.4. Initializer

In [ ]:
init = tf.global_variables_initializer()

### 2.4. Run the session
- 앞서 만든 graph, operator 등을 돌리는 과정

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(init)

In [ ]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_op, feed_dict={X: batch_xs, Y: batch_ys})
    if (i + 1) % 500 == 0:
        val_acc = sess.run(accuracy, feed_dict={X: X_val, Y: Y_val})
        print("[%d step] validation accuracy: %g" % (i + 1, val_acc))

In [ ]:
print(sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

In [ ]:
sess.close()